In [11]:
# Anpassung der Bibliothek Pybtex für den APA-Style

import re
import io
from pybtex.database import parse_string
from pybtex.plugin import find_plugin
from pybtex.style.formatting import BaseStyle, toplevel
from pybtex.style.template import (
    field, first_of, href, join, optional, optional_field, sentence, tag,
    together, words, node, FieldIsMissing
)
from pybtex import richtext
from pybtex.richtext import Text, Symbol

firstlast = find_plugin('pybtex.style.names', 'firstlast')()

def format_pages(text):
    dash_re = re.compile(r'-+')
    pages = Text(Symbol('ndash')).join(text.split(dash_re))
    if re.search('[-‒–—―]', str(text)):
        return Text("pp.", Symbol('nbsp'), pages)
    return Text("p.", Symbol('nbsp'), pages)

pages = field('pages', apply_func=format_pages)
date = words[field('year'), optional[", ", field('month')]]

@node
def apa_names(children, context, role, **kwargs):
    """
    Returns formatted names as an APA compliant reference list citation.
    """
    assert not children

    try:
        persons = context['entry'].persons[role]
    except KeyError:
        raise FieldIsMissing(role, context['entry'])

    style = context['style']

    if len(persons) > 20:
        formatted_names = [style.format_name(
            person, style.abbreviate_names) for person in persons[:20]]
        formatted_names += [richtext.Text("et al.")]
        return join(sep=', ')[formatted_names].format_data(context)
    else:
        formatted_names = [style.format_name(
            person, style.abbreviate_names) for person in persons]
        return join(sep=', ', sep2=' & ', last_sep=', & ')[
            formatted_names].format_data(context)

@node
def editor_names(children, context, with_suffix=True, **kwargs):
    """
    Returns formatted editor names for inbook.
    """
    assert not children

    try:
        editors = context['entry'].persons['editor']
    except KeyError:
        raise FieldIsMissing('editor', context['entry'])

    formatted_names = [
        firstlast.format(editor, True) for editor in editors]

    if with_suffix:
        return words[
            join(sep=', ', sep2=', & ', last_sep=', & ')[formatted_names],
            "(Eds.)" if len(editors) > 1 else "(Ed.)"
        ].format_data(context)

    return join(sep=', ', sep2=', & ', last_sep=', & ')[
        formatted_names
    ].format_data(context)

class APAStyle(BaseStyle):
    name = 'apa7'
    default_name_style = 'lastfirst'
    default_sorting_style = 'author_year_title'
    default_label_style = 'apa7'

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.abbreviate_names = True

    def format_names(self, role, as_sentence=True):
        formatted_names = apa_names(role)
        if as_sentence:
            return sentence(capfirst=False)[formatted_names]
        else:
            return formatted_names

    def format_editor(self, e, as_sentence=True):
        editors = self.format_names('editor', as_sentence=False)
        if 'editor' not in e.persons:
            # when parsing the template, a FieldIsMissing exception
            # will be thrown anyway; no need to do anything now,
            # just return the template that will throw the exception
            return optional[field('editor')]
        if len(e.persons['editor']) > 1:
            word = '(Eds.)'
        else:
            word = '(Ed.)'
        result = join(sep=' ')[editors, word]
        if as_sentence:
            return sentence[result]
        else:
            return result

    def format_volume(self, e, for_article=False):
        prefix = "Vol."
        if for_article:
            return join[
                tag('em')[field('volume')],
                optional['(', field('number'), ')'],
            ]
        else:
            return optional[together[prefix, field('volume')]]

    def format_number(self, e):
        prefix = "No."
        return optional[together[prefix, field('number')]]

    def format_edition(self, e):
        suffix = "ed."
        return optional[together[field('edition'),suffix]]
        
    def format_pages(self, e):
        prefix = "pp."
        return optional[together[prefix,field('pages')]]

    def format_title(self, e, which_field, as_sentence=True):
        formatted_title = field(
            which_field, apply_func=lambda text: text.capitalize()
        )
        if as_sentence:
            return sentence[formatted_title]
        else:
            return formatted_title

    def format_btitle(self, e, which_field, as_sentence=True):
        formatted_title = tag('em')[field(which_field)]
        if as_sentence:
            return sentence[formatted_title]
        else:
            return formatted_title

    def format_web_refs(self, e):
        return sentence(add_period=False)[
            optional[self.format_url(e)],
            optional[self.format_eprint(e)],
            optional[self.format_pubmed(e)],
            optional[self.format_doi(e)],
        ]

    def format_url(self, e):
        return words[
            'URL:',
            href[
                field('url', raw=True),
                field('url', raw=True)
            ]
        ]

    def format_pubmed(self, e):
        return href[
            join[
                'https://www.ncbi.nlm.nih.gov/pubmed/',
                field('pubmed', raw=True)
            ],
            join[
                'PMID:',
                field('pubmed', raw=True)
            ]
        ]

    def format_doi(self, e):
        return href[
            join[
                'https://doi.org/',
                field('doi', raw=True)
            ],
            join[
                'doi:',
                field('doi', raw=True)
            ]
        ]

    def format_eprint(self, e):
        return href[
            join[
                'https://arxiv.org/abs/',
                field('eprint', raw=True)
            ],
            join[
                'arXiv:',
                field('eprint', raw=True)
            ]
        ]

    def format_date(self, e):
        return sentence[
            join["(", first_of[optional[date], "n.d."], ")"]
        ]

    def get_article_template(self, e):
        volume_and_pages = first_of[
            optional[
                join(sep=', ')[
                    self.format_volume(e, for_article=True),
                    optional['pp. ',field('pages')]
                ]
            ],
            pages,
        ]
        return toplevel[
            self.format_names('author'),
            sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
            self.format_title(e, 'title'),
            sentence[
                tag('em')[field('journal')],
                optional[volume_and_pages],
            ],
            sentence[optional_field('note')],
            self.format_web_refs(e),
        ]

    def get_book_template(self, e):
        #print(e.fields)
        #print(e.persons)
        if 'series' in e.fields or 'edition' in e.fields or 'volume' in e.fields or 'number' in e.fields or 'pages' in e.fields or 'editor' in e.persons:
            volume_and_pages = join['(', join(sep=', ')[
                                            self.format_editor(e, as_sentence=False),
                                            optional_field('series'),
                                            self.format_edition(e),
                                            self.format_volume(e),
                                            self.format_number(e),
                                            self.format_pages(e)]
                                    ,')'] 
        else: 
            volume_and_pages = None
        return toplevel[
            self.format_names('author'),
            sentence[join['(',field('year'),')']],
            sentence(sep=' ')[
                field('title'),
                volume_and_pages
            ],
            sentence()[field('publisher'),],
            sentence[optional_field('address')],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_booklet_template(self, e):
        if 'series' in e.fields or 'volume' in e.fields or 'number' in e.fields or 'pages' in e.fields or 'editor' in e.persons:
            volume_number_pages = join['(', join(sep=', ')[
                                            self.format_editor(e, as_sentence=False),
                                            optional_field('series'),
                                            self.format_volume(e),
                                            self.format_number(e),
                                            self.format_pages(e)]
                                    ,')'] 
        else: 
            volume_number_pages = None
        return toplevel[
            self.format_names('author'),
            sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
            sentence(sep=' ')[
                field('title'), '[Brochure]', volume_number_pages  
            ],
            sentence[optional_field('organization')],
            sentence[optional_field('howpublished')],
            sentence[optional_field('address')],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_inbook_template(self, e):
        if 'editor' in e.persons:
            editors = join[self.format_editor(e, as_sentence=False),','] 
        else: 
            editors = None
        if 'series' in e.fields or 'edition' in e.fields or 'volume' in e.fields or 'number' in e.fields or 'pages' in e.fields:
            volume_and_pages = join['(', join(sep=', ')[
                                            optional_field('series'),
                                            self.format_edition(e),
                                            self.format_volume(e),
                                            self.format_number(e),
                                            self.format_pages(e)]
                                    ,')'] 
        else: 
            volume_and_pages = None
        return toplevel[
            self.format_names('author'),
            sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
            sentence(sep=' ')[optional['Chapter ',field('chapter'), ':'], self.format_title(e, 'title')],
            optional[
                sentence(sep=' ')["In", editors,
                self.format_btitle(e, 'booktitle', as_sentence=False),
                volume_and_pages
            ]],
            sentence()[field('publisher'),],
            sentence[optional_field('address')],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]
    
    def get_incollection_template(self, e):
        if 'editor' in e.persons:
            editors = join[self.format_editor(e, as_sentence=False),','] 
        else: 
            editors = None
        if 'series' in e.fields or 'edition' in e.fields or 'volume' in e.fields or 'number' in e.fields or 'pages' in e.fields:
            volume_and_pages = join['(', join(sep=', ')[
                                            optional_field('series'),
                                            self.format_edition(e),
                                            self.format_volume(e),
                                            self.format_number(e),
                                            self.format_pages(e)]
                                    ,')'] 
        else: 
            volume_and_pages = None
        return toplevel[
            self.format_names('author'),
            sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
            sentence[self.format_title(e, 'title')],
            optional[
                sentence(sep=' ')["In", editors,
                self.format_btitle(e, 'booktitle', as_sentence=False),
                volume_and_pages
            ]],
            sentence()[field('publisher'),],
            sentence[optional_field('address')],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_inproceedings_template(self, e):
        if 'editor' in e.persons:
            editors = join[self.format_editor(e, as_sentence=False),','] 
        else: 
            editors = None
        if 'series' in e.fields or 'edition' in e.fields or 'volume' in e.fields or 'number' in e.fields or 'pages' in e.fields:
            volume_and_pages = join['(', join(sep=', ')[
                                            optional_field('series'),
                                            self.format_edition(e),
                                            self.format_volume(e),
                                            self.format_number(e),
                                            self.format_pages(e)]
                                    ,')'] 
        else: 
            volume_and_pages = None
        return toplevel[
            self.format_names('author'),
            sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
            sentence[self.format_title(e, 'title')],
            sentence(sep=' ')["In", editors,
                self.format_btitle(e, 'booktitle', as_sentence=False),
                volume_and_pages
            ],
            sentence[optional_field('publisher'),],
            sentence[optional_field('organization')],
            sentence[optional_field('address')],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_manual_template(self, e):
        return toplevel[
            optional[self.format_names('author')],
            sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
            optional[sentence(sep=' ')[
                field('title'),
                optional['(',field('edition'), ' ed.',')']
                ]
            ],
            sentence[optional_field('organization')],
            sentence[optional_field('address')],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_misc_template(self, e):
        if 'author' in e.persons:
            return toplevel[
                self.format_names('author'),
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                optional[self.format_btitle(e, 'title')],
                sentence[optional_field('note')],
            ]
        else:
            return toplevel[
                optional[self.format_btitle(e, 'title')],
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]

    def get_phdthesis_template(self, e):
        return toplevel[
            self.format_names('author'),
            sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
            sentence(sep=' ')[
                self.format_btitle(e, 'title', as_sentence=False),
                #optional['(',field('type'),')'],
                sentence(sep='')['[','Doctoral dissertation, ',
                field('school'),']']
            ],
            sentence[optional_field('address')],
            sentence[optional[pages]],
            sentence[optional_field('note')],
            self.format_web_refs(e)
        ]

    def get_proceedings_template(self, e):
        if 'series' in e.fields or 'edition' in e.fields or 'volume' in e.fields or 'number' in e.fields or 'pages' in e.fields:
            volume_and_pages = join['(', join(sep=', ')[
                                            optional_field('series'),
                                            self.format_edition(e),
                                            self.format_volume(e),
                                            self.format_number(e),
                                            self.format_pages(e)]
                                    ,')'] 
        else: 
            volume_and_pages = None
        if 'editor' in e.persons:
            return toplevel[
                self.format_editor(e),
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                sentence(sep=' ')[
                    self.format_btitle(e, 'title', as_sentence=False),
                    volume_and_pages
                ],
                sentence[optional_field('publisher')],
                sentence[optional_field('organization')],
                sentence[optional_field('address')],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]
        else:
            return toplevel[
                sentence(sep=' ')[
                    field('title'),
                    volume_and_pages
                ],
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                sentence[optional_field('publisher')],
                #sentence[optional_field('series')],
                sentence[optional_field('organization')],
                sentence[optional_field('address')],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]

    def get_techreport_template(self, e):
        if 'author' in e.persons:
            return toplevel[
                self.format_names('author'),
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                self.format_btitle(e, 'title'),
                sentence[optional_field('number')],
                sentence[field('institution')],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]
        else:
            return toplevel[
                self.format_btitle(e, 'title'),
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                sentence[optional_field('number')],
                sentence[field('institution')],
                sentence[optional_field('note')],
                self.format_web_refs(e)
            ]

    def get_unpublished_template(self, e):
        if 'author' in e.persons:
            return toplevel[
                self.format_names('author'),
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                self.format_btitle(e, 'title'),
                sentence[field('note')],
                self.format_web_refs(e)
            ]
        else:
            return toplevel[
                self.format_btitle(e, 'title'),
                sentence[join['(',join(sep=', ')[field('year'),optional_field('month')],')']],
                sentence[field('note')],
                self.format_web_refs(e)
            ]


In [12]:
# Beispiel-BibTeX-Code
bibtex_code = """
@proceedings{conference2023proceedings,
  title = {Proceedings of the International Conference on Research Methods},
  month = {April},
  year = {2023},
  volume = {5},
  number = {2},
  series = {Conference Proceedings Series},
  note = {Conference held at New York, NY},
  address = {New York, NY},
  publisher = {Academic Press},
  organization = {International Conference on Research Methods},
  editor = {Johnson, Alice and Brown, Robert}
}


"""

# BibTeX-Code parsen
bib_data = parse_string(bibtex_code, 'bibtex')
#parser = bibtex.Parser()
#bib_data = parser.parse_file("mybib.bib")

# Erstellen Sie eine Instanz der APA-Stilklasse
apa_style = APAStyle()
plain_backend = find_plugin('pybtex.backends', 'text')()

# Einträge formatieren
formatted_entries = apa_style.format_entries(bib_data.entries.values())

# Custom Backend zum Erfassen der Ausgabe in einem String
class StringBackend(plain_backend.__class__):
    def __init__(self):
        super().__init__()
        self.output = io.StringIO()

    def write_entry(self, label, text):
        self.output.write(text + "\n")

    def get_output(self):
        return self.output.getvalue()

# Custom Backend verwenden
string_backend = StringBackend()
try:
    for entry in formatted_entries:
        string_backend.write_entry(entry.key, entry.text.render(string_backend))
except FieldIsMissing as ex:
    print(ex)

# Formatierten String erhalten
formatted_string = string_backend.get_output()
print(formatted_string)

Johnson, A. & Brown, R. (Eds.). (2023, April). Proceedings of the International Conference on Research Methods (Conference Proceedings Series, Vol. 5, No. 2). Academic Press. International Conference on Research Methods. New York, NY. Conference held at New York, NY.

